# Graph

In reality, there is no reason to create a graph with vertex and edge data structures because Azure and AWS both have graph databases: Azure has [CosmosDb](https://azure.microsoft.com/en-us/services/cosmos-db/) and AWS has [Neptune](https://aws.amazon.com/neptune/).  However, my purpose is to explore not only how to implement a potential graph application, but also to explore Rete algorithms a bit.  How do they work?

## Implementing Vertices and Edges

A few years ago I wrote a graph service around the Titan database.  One thing that impressed me was the lookup time on the database.  Once I learned it, I found [Gremlin](https://en.wikipedia.org/wiki/Gremlin_(programming_language) to be incredibly powerful.  For my purposes, I want the graph to be directional and I want to maintain a dictionary on both vertices and edges for storing information.  For Rete graphs, the graph should be acyclic, but I have not put in any checks for that in my graph.

First thing, I want a generator for vertex and edge ids and I need them to be unique.

In [1]:
import uuid

def new_vertex_id():
    return uuid.uuid4()

def edge_id(vertex1id, vertex2id):
    return "{}.{}".format(str(vertex1id), str(vertex2id))

Create the graph classes.  Because Python (in notebooks - we have no import unless we dump code into a py file) requires us to define things in order, we are going to use inheritance to make sure we can get the proper interdependence to make the graph lean and at least somewhat performant.

In [2]:
class Base:
    def __init__(self):
        self.data = {}

    def insert_data(self, name, value):
        # We will let python generate the exception if the name already in dictionary
        self.data[name] = value
        
    def get_data(self, name):
        if name in self.data:
            return self.data[name]
        return None

In [3]:
class VertexBase(Base):
    # Constructor
    def __init__(self):
        super().__init__()
        self.id = new_vertex_id()
        
    def get_id(self):
        return self.id
    
    def print(self):
        print("Vertex (ID={})".format(self.id))
        print("Data: {}".format(self.data))

In [4]:
class EdgeBase(Base):
    def __init__(self, v1, v2):
        super().__init__()
        self.v1 = v1.get_id()
        self.v2 = v2.get_id()
        
    def get_id(self):
        return edge_id(self.v1, self.v2)
    
    def print(self):
        print("Edge (ID={})".format(self.get_id()))
        print("Data: {}".format(self.data))

In [5]:
class Graph:
    # Constructor
    def __init__(self):
        self.vertices = {}
        self.edges = {}
    
    def add_vertex(self, vtx):
        self.vertices[vtx.get_id()] = vtx
        return vtx
    
    def add_edge(self, edg):
        self.edges[edg.get_id()] = edg
        return edg
    
    def get_vertex(self, vtxid):
        if vtxid in self.vertices:
            return self.vertices[vtxid]
        return None
    
    def get_edge(self, edgid):
        if edgid in self.edges:
            return self.edges[edgid]
        return None

In [6]:
class Vertex(VertexBase):
    def __init__(self):
        super().__init__()
        self.edges = []
        self.iv = 0
    
    def add_edge(self, edge):
        self.edges.append(edge.get_id())
        
    def get_edge(self, graph):
        if len(self.edges) <= self.iv:
            return None
        self.iv = self.iv + 1
        return graph.get_edge(self.edges[self.iv-1])
    
    def get_first_edge(self, graph):
        self.iv = 0
        return self.get_edge(graph)
    
    def number_edges(self):
        return len(self.edges)
    
    def print(self):
        print("VERTEX")
        super().print()
        print("Edges")
        for i in range(len(self.edges)):
            print(self.edges[i])
        print("")

In [7]:
class Edge(EdgeBase):
    def __init__(self, v1, v2):
        super().__init__(v1, v2)

    def get_vertex1(self, graph):
        return graph.get_vertex(self.v1)
    
    def get_vertex2(self, graph):
        return graph.get_vertex(self.v2)
    
    def print(self):
        print("EDGE")
        super().print()
        print("Vertices")
        print("Vertex 1: {}".format(self.v1))
        print("Vertex 2: {}".format(self.v2))
        print("")

When an edge is created, several relations need to be taken care of at one time.  Edges should always be created with the following routine.

In [8]:
def create_edge(graph, vertex1, vertex2):
    e = Edge(vertex1, vertex2)
    vertex1.add_edge(e)
    vertex2.add_edge(e)
    graph.add_edge(e)
    return e

We don't do a fancy job of viewing the graph.  However, our graphs will be small enought we can just print them out.

In [9]:
def dump_graph(graph, v, visited = None):
    if visited is None:
        visited = []
    v.print()
    e = v.get_first_edge(graph)
    while e is not None:
        if e.get_id() not in visited:
            visited.append(e.get_id())
            e.print()
            vend = e.get_vertex2(graph)
            dump_graph(graph, vend, visited)
        e = v.get_edge(graph)

Just a little test graph.  Create a simple graph.

In [10]:
g = Graph()
v1 = g.add_vertex(Vertex())
v1.insert_data('test', 'Can I add a test data')
v2 = g.add_vertex(Vertex())
v2.insert_data('test', 'Perhaps?')
v3 = g.add_vertex(Vertex())
v3.insert_data('test', 'We will connect here')
e1 = create_edge(g, v1, v2)
e2 = create_edge(g, v2, v3)

dump_graph(g, v1)

VERTEX
Vertex (ID=3953b866-b772-43ed-84ac-46cd678dc238)
Data: {'test': 'Can I add a test data'}
Edges
3953b866-b772-43ed-84ac-46cd678dc238.d40be806-ea19-4c87-9b8a-0c820c2887c9

EDGE
Edge (ID=3953b866-b772-43ed-84ac-46cd678dc238.d40be806-ea19-4c87-9b8a-0c820c2887c9)
Data: {}
Vertices
Vertex 1: 3953b866-b772-43ed-84ac-46cd678dc238
Vertex 2: d40be806-ea19-4c87-9b8a-0c820c2887c9

VERTEX
Vertex (ID=d40be806-ea19-4c87-9b8a-0c820c2887c9)
Data: {'test': 'Perhaps?'}
Edges
3953b866-b772-43ed-84ac-46cd678dc238.d40be806-ea19-4c87-9b8a-0c820c2887c9
d40be806-ea19-4c87-9b8a-0c820c2887c9.862c9ca4-1510-4aef-8cc5-7767da488034

EDGE
Edge (ID=d40be806-ea19-4c87-9b8a-0c820c2887c9.862c9ca4-1510-4aef-8cc5-7767da488034)
Data: {}
Vertices
Vertex 1: d40be806-ea19-4c87-9b8a-0c820c2887c9
Vertex 2: 862c9ca4-1510-4aef-8cc5-7767da488034

VERTEX
Vertex (ID=862c9ca4-1510-4aef-8cc5-7767da488034)
Data: {'test': 'We will connect here'}
Edges
d40be806-ea19-4c87-9b8a-0c820c2887c9.862c9ca4-1510-4aef-8cc5-7767da488034



Using the first vertex, walk through the graph.

In [11]:
vx1 = g.get_vertex(v1.get_id())
print(vx1.get_data('test'))

Can I add a test data


In [12]:
ed1 = vx1.get_first_edge(g)
mid_vtx = ed1.get_vertex2(g)
print(mid_vtx.get_data('test'))
ed2 = mid_vtx.get_first_edge(g)
# Don't walk back on the edge we just came from
if ed2.get_id() == ed1.get_id():
    ed2 = mid_vtx.get_edge(g)
end_vtx = ed2.get_vertex2(g)
print(end_vtx.get_data('test'))

Perhaps?
We will connect here


# Rete Algorithm

The [Rete Algorithm](https://en.wikipedia.org/wiki/Rete_algorithm) drives rules-based systems.  Apparently systems like Drupal and others use the Rete algorithm.  To the best of my understanding, the Rete algorithm works like walking through a directed graph.  Because it is a graph, it allows us to walk back to a rule and try other options.

Unsurprisingly, the Rete Algorithm looks a lot like the Themis patent that we generated for walking through APIs.

We are going to use a visited list for edges we have already visited.  In that way, we won't walk down the same path again.  Let's set up a set of rules with just some numbers.  Here are the rules - for some number x (let x = 85);

1) If x > 0, then go to rule 2.  If x < 2000, then go to rule 4.  (Both are true)  
2) If x > 10, then go to rule 3. (passes)  
3) If x < 80, then we are done.  (does not pass)  
4) If x < 100, then go to rule 5. (passes)  
5) If x > 80, then we are done.  (We pass)  

Set up the graph.

In [13]:
g = Graph()
start = g.add_vertex(Vertex())
start.insert_data(1, "START")
rule2 = g.add_vertex(Vertex())

# The first rule off of the start is x > 0
e1 = create_edge(g, start, rule2)
e1.insert_data(1, ['x', '>', 0])
rule4 = g.add_vertex(Vertex())

# The other rule off of the start is x < 2000
e2 = create_edge(g, start, rule4)
e2.insert_data(1, ['x', '<', 2000])
rule3 = g.add_vertex(Vertex())

# Rule 2 above is x > 10
e3 = create_edge(g, rule2, rule3)
e3.insert_data(1, ['x', '>', 10])
end_rule1 = g.add_vertex(Vertex())
end_rule1.insert_data(1, 'TERMINAL')

# This is listed as rule3 - it is our first choice where we successfully pass a set of rules.
e4 = create_edge(g, rule3, end_rule1)
e4.insert_data(1, ['x', '<', 80])

# This is listed as rule 4 going to rule 5
rule5 = g.add_vertex(Vertex())
e5 = create_edge(g, rule4, rule5)
e5.insert_data(1, ['x', '<', 100])

end_rule2 = g.add_vertex(Vertex())
end_rule2.insert_data(1, 'TERMINAL')
e6 = create_edge(g, rule5, end_rule2)
e6.insert_data(1, ['x', '>', 80])

dump_graph(g, start)


VERTEX
Vertex (ID=802461cf-f414-4e6e-8236-93869b675f39)
Data: {1: 'START'}
Edges
802461cf-f414-4e6e-8236-93869b675f39.9f3b869e-ad01-4fc8-bdc6-3316d939a11a
802461cf-f414-4e6e-8236-93869b675f39.4b08cf09-e0b7-44cd-954a-8951685c20d5

EDGE
Edge (ID=802461cf-f414-4e6e-8236-93869b675f39.9f3b869e-ad01-4fc8-bdc6-3316d939a11a)
Data: {1: ['x', '>', 0]}
Vertices
Vertex 1: 802461cf-f414-4e6e-8236-93869b675f39
Vertex 2: 9f3b869e-ad01-4fc8-bdc6-3316d939a11a

VERTEX
Vertex (ID=9f3b869e-ad01-4fc8-bdc6-3316d939a11a)
Data: {}
Edges
802461cf-f414-4e6e-8236-93869b675f39.9f3b869e-ad01-4fc8-bdc6-3316d939a11a
9f3b869e-ad01-4fc8-bdc6-3316d939a11a.18ea3068-57f5-4fe7-a3bf-167209912405

EDGE
Edge (ID=9f3b869e-ad01-4fc8-bdc6-3316d939a11a.18ea3068-57f5-4fe7-a3bf-167209912405)
Data: {1: ['x', '>', 10]}
Vertices
Vertex 1: 9f3b869e-ad01-4fc8-bdc6-3316d939a11a
Vertex 2: 18ea3068-57f5-4fe7-a3bf-167209912405

VERTEX
Vertex (ID=18ea3068-57f5-4fe7-a3bf-167209912405)
Data: {}
Edges
9f3b869e-ad01-4fc8-bdc6-3316d939a11a.18ea3

Set up the rule set using our graph.  There are some gotchas when implementing a Rete algorithm.  One is that you don't want to get stuck walking up rule paths which have already been visited.  Therefore, like the [Dijkstra algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) we will keep a list of visited edges.  The other thing that seems like a good idea to me is to be able to quickly pop back through the path we just walked till we find another edge off a vertex that has not been visited and can test that rule.  Our test value is 85.

In [14]:
import queue

# Test value
x = 85

# Edges visited
visited = []

# Path - These are ordered edges of where we just came from.
path = queue.LifoQueue()

# Define a function for testing our rules.  Very simple - just less than or greater than.
def apply_rule(data, x):
    print('RULE : {} -> {} {} {}'.format(data, x, data[1], data[2]))
    if data[1] == '>':
        return x > data[2]
    elif data[1] == '<':
        return x < data[2]

# Try the first rule (edge) from the start.
start_choice_1 = start.get_first_edge(g)

# Since we haven't visited any edges, go with this one...
visited.append(start_choice_1.get_id())
path.put(start_choice_1.get_id())

# Print the rule check.
print(apply_rule(start_choice_1.get_data(1), x))


RULE : ['x', '>', 0] -> 85 > 0
True


In [15]:
# Now get the other vertex because we passed the rule
vtx = start_choice_1.get_vertex2(g)

# Get the next unvisited edge.
next_choice = vtx.get_first_edge(g)
if next_choice.get_id() in visited:
    next_choice = vtx.get_edge(g)
    
# Ideally, we check for None (no more rules), but we are just testing the concept here...

# Mark visited, add to path
visited.append(next_choice.get_id())
path.put(next_choice.get_id())

# Print the rule check.
print(apply_rule(next_choice.get_data(1), x))

RULE : ['x', '>', 10] -> 85 > 10
True


In [16]:
# Get the other vertex because we passed the rule
vtx = next_choice.get_vertex2(g)

# Get the next unvisited edge
first_way_out = vtx.get_first_edge(g)
if first_way_out.get_id() in visited:
    first_way_out = vtx.get_edge(g)
    
# Again, we should check for None, but I know I have one - just running the graph

# Mark visited, add to path.
visited.append(first_way_out.get_id())
path.put(first_way_out.get_id())

# Print the rule - did we pass?
print('TERMINAL ? = {}'.format(first_way_out.get_vertex2(g).get_data(1)))
print(apply_rule(first_way_out.get_data(1), x))

TERMINAL ? = TERMINAL
RULE : ['x', '<', 80] -> 85 < 80
False


We verified that this was the final rule, but we didn't pass the rule.  Now we back up our path (in the LIFO queue) until we find another path.

In [17]:
# Pop the last edge we put on because we added the edge to the terminal vertex before we checked
# that we failed the rule
path.get()

# We can use out path to back up until we find an edge we have not visisted
while not path.empty():
    prev_edge_id = path.get()
    prev_edge = g.get_edge(prev_edge_id)
    fvertex = prev_edge.get_vertex1(g)
    anedge = fvertex.get_first_edge(g)
        
    while anedge is not None and anedge.get_id() in visited:
        # Get the next edge
        anedge = fvertex.get_edge(g)
        
    if anedge is None:
        print('We do not have a path to go down from here, go another')
    else:
        print('We have an edge to check with ID {}'.format(anedge.get_id()))
        break

We do not have a path to go down from here, go another
We have an edge to check with ID 802461cf-f414-4e6e-8236-93869b675f39.4b08cf09-e0b7-44cd-954a-8951685c20d5


We know from how we constructed the rule set that we had no other paths to follow until we got back to the very first rule.  Because of that, we know that we have just cleared out path - that whole rule path was invalidated on the last rule.  We are going to have to try another branch in our rule set.  Also, since our path is cleared, we are back to the very first (entry or START) vertex.

In [18]:
# Note that we should have emptied the path queue - we know we had to go back to the start.
# Let's check it
print('Path Empty? {}'.format(path.empty()))
print('Start Vertex? {}'.format(anedge.get_vertex1(g).get_data(1)))

Path Empty? True
Start Vertex? START


In [19]:
# We have an edge, add to visited and path, then let's check the rule
visited.append(anedge.get_id())
path.put(anedge.get_id())
print(apply_rule(anedge.get_data(1), x))

RULE : ['x', '<', 2000] -> 85 < 2000
True


In [20]:
# Try the next rule
vtxx = anedge.get_vertex2(g)
second_way_out = vtxx.get_first_edge(g)
if second_way_out.get_id() in visited:
    second_way_out = vtxx.get_edge(g)

# Bookkeeping - add to visited and path
visited.append(second_way_out.get_id())
path.put(second_way_out.get_id())

# Terminal vertex, did we pass the last rule?
print('TERMINAL ? = {}'.format(second_way_out.get_vertex2(g).get_data(1)))
print(apply_rule(second_way_out.get_data(1), x))


TERMINAL ? = None
RULE : ['x', '<', 100] -> 85 < 100
True


And since we passed this terminal path, we now have a working set of rules - the ids of those rules are in our path which we print below. For completness, print the rules that were successfully applied.

In [21]:
lpath = list(path.queue)
for l in lpath:
    e = g.get_edge(l)
    print(apply_rule(e.get_data(1), x))

RULE : ['x', '<', 2000] -> 85 < 2000
True
RULE : ['x', '<', 100] -> 85 < 100
True


 As I understand it, this is the Rete algorithm implemented on a directed (acyclic) graph.